In [1]:
import wfdb
import matplotlib.pyplot as plt
import numpy as np
from hrv.filters import quotient, moving_median
from scipy import interpolate
from tqdm import tqdm
import os
from numpy import genfromtxt

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [2]:
#define functions
def create_time_info(rri):
    # Return the cumulative sum of the RRI durations in the array
    rri_time = np.cumsum(rri) / 1000.0  # make it seconds
#     print("Cumulative time: " + str(rri_time))
#     print("RRI_time[0]: " + str(rri_time[0]))
    return rri_time - rri_time[0]   # force it to start at zero

def create_interp_time(rri, fs):
    time_rri = create_time_info(rri)
#     print("cummulative time from 0 in seconds: " + str(time_rri))
    #Values are generated within the half-open interval [start, stop) (in other words, the interval including start but excluding stop)
    return np.arange(0, time_rri[-1], 1 / float(fs))

def interp_cubic_spline(rri, fs):
    # create time scale from 0
    time_rri = create_time_info(rri)
#     print("cummulative time from 0 in seconds: " + str(time_rri))
    # create time scale with new sampling rate
    time_rri_interp = create_interp_time(rri, fs)
#     print("cummulative time after interpolation with new f: " + str(time_rri_interp))
    tck = interpolate.splrep(time_rri, rri, s=0)
    rri_interp = interpolate.splev(time_rri_interp, tck, der=0)
    return time_rri_interp, rri_interp

def interp_cubic_spline_qrs(qrs_index, qrs_amp, fs):
    time_qrs = qrs_index / float(FS)
    time_qrs = time_qrs - time_qrs[0]
    time_qrs_interp = np.arange(0, time_qrs[-1], 1/float(fs))
    tck = interpolate.splrep(time_qrs, qrs_amp, s=0)
    qrs_interp = interpolate.splev(time_qrs_interp, tck, der=0)
    return time_qrs_interp, qrs_interp

# get the amplitude of local R-peak,  Correct the peaks shifting them to local maxima
# min_bpm = 25
# max_bpm = 240, justify 
# Use the maximum possible bpm as the search radius,
# interval  = int(record.fs * 60 / max_bpm)
# INPUT: ECG, QRS - list of samples labled "QRS"
# OUTPUT: a corresponding array of R-peak amplitude

# use wfdb to test output of demo19 with this output from get_qrs_amp
def get_qrs_amp(ecg, qrs):
    interval = int(FS * 0.250)
    qrs_amp = []
    for index in range(len(qrs)):
        curr_qrs = qrs[index]
        # Find local R peak
        amp = np.max(ecg[curr_qrs-interval:curr_qrs+interval])
        qrs_amp.append(amp)

    return qrs_amp

In [3]:
# Read 5 patients dataset
data_path = './apnea-ecg-spaches/'

# test_data_name = ['patient1', 'patient2', 'patient3', 'patient4', 'patient5',]
test_data_name = ['patient5']

# age = [63, 50, 68, 55, 49]
age = [49]
# sex = [1, 0, 0, 0, 0]
sex = [0]

In [8]:
age = [63, 50, 68, 55, 49]
print("Mean: " + np.mean(np.array(age)).astype(str))
print("std: " + np.std(np.array(age)).astype(str))

Mean: 57.0
std: 7.402702209328699


In [6]:
# TEST 
# name duration AHI
# patient1 440.5 1.4
# patient2 456 0.1
# patient3 295.9 47.4
# patient4 587.9 6.5
# patient5 417 12.4
duration = [440.5, 456, 295.9,587.9,417]
AHI = [1.4,0.1,47.4,6.5,12.4]
print("Mean: " + np.mean(np.array(duration)).astype(str))
print("std: " + np.std(np.array(duration)).astype(str))

print("Mean AHI: " + np.mean(np.array(AHI)).astype(str))
print("std AHI: " + np.std(np.array(AHI)).astype(str))

Mean: 439.46000000000004
std: 93.19121417816166
Mean AHI: 13.559999999999999
std AHI: 17.466379132493376


In [91]:
# define variables
# ECG_RAW_FREQUENCY
FS = 100.0
MARGIN = 10
FS_INTP = 4
MAX_HR = 300.0
MIN_HR = 20.0

MIN_RRI = 1.0 / (MAX_HR / 60.0) * 1000 #20ms
MAX_RRI = 1.0 / (MIN_HR / 60.0) * 1000 #3000ms
train_input_array = []
train_label_array = []
# create test input
test_input_array = []
test_label_array = []

In [92]:
import pandas as pd
df=pd.read_csv('apnea-ecg-spaches/patient5_apn.csv', sep=',',header=None)
# df=pd.read_csv('apnea-ecg-spaches/patient4_apn.csv', sep=',',header=None)
# df=pd.read_csv('apnea-ecg-spaches/patient3_apn.csv', sep=',',header=None)
# df=pd.read_csv('apnea-ecg-spaches/patient2_apn.csv', sep=',',header=None)
# df=pd.read_csv('apnea-ecg-spaches/patient1_apn.csv', sep=',',header=None)
win_num = df.values
len(win_num)
win_num[0][0]

'A'

In [93]:
df1=pd.read_csv('apnea-ecg-spaches/patient5_qrs_i_raw.csv', sep=',',header=None)
qrs = df1.values
len(qrs)
# qrs[21]

5162

In [94]:
df2=pd.read_csv('apnea-ecg-spaches/patient5_qrs_amp.csv', sep=',',header=None)
qrs_amp_raw = df2.values
qrs_amp_raw
# len(qrs_amp)
qrs_amp_raw[21]

array([0.49669078])

In [110]:
for data_index in range(len(test_data_name)):
    print (test_data_name[data_index])
    df=pd.read_csv('./apnea-ecg-spaches/' + test_data_name[data_index] + '_apn.csv', sep=',',header=None)
    win_num = df.values
    df1=pd.read_csv('./apnea-ecg-spaches/' + test_data_name[data_index] + '_qrs_i_raw.csv', sep=',',header=None)
    qrs = df1.values
    df2=pd.read_csv('./apnea-ecg-spaches/' + test_data_name[data_index] + '_qrs_amp.csv', sep=',',header=None)
    qrs_amp_raw = df2.values
    
    for index in tqdm(range(0, len(win_num))):
        samp_from = index * 60 * FS # 60 seconds
        samp_to = samp_from + 60 * FS  # 60 seconds

        sub_qrs =  np.where(np.logical_and(qrs>=samp_from, qrs<=samp_to-1))
        qrs_ann = qrs[sub_qrs]
        qrs_amp = qrs_amp_raw[sub_qrs]

        apn_ann = win_num[index]

        rri = np.diff(qrs_ann)
        rri_ms = rri.astype('float') / FS * 1000.0
        print(qrs_ann)
        print(qrs_amp)
        print(rri)
        print(rri_ms)
        try:
                rri_filt = moving_median(rri_ms)
                print(rri_filt)

                if len(rri_filt) > 5 and (np.min(rri_filt) >= MIN_RRI and np.max(rri_filt) <= MAX_RRI):
                    time_intp, rri_intp = interp_cubic_spline(rri_filt, FS_INTP)
                    qrs_time_intp, qrs_intp = interp_cubic_spline_qrs(qrs_ann, qrs_amp, FS_INTP)
                    rri_intp = rri_intp[(time_intp >= MARGIN) & (time_intp < (60+MARGIN))]
                    qrs_intp = qrs_intp[(qrs_time_intp >= MARGIN) & (qrs_time_intp < (60 + MARGIN))]
                    #time_intp = time_intp[(time_intp >= MARGIN) & (time_intp < (60+MARGIN))]
                    
#                     if len(rri_intp) != (FS_INTP * 60):
#                         skip = 1
#                     else:
#                         skip = 0

#                     if skip == 0:
                    print("OK")
                    rri_intp = rri_intp - np.mean(rri_intp)
                    qrs_intp = qrs_intp - np.mean(qrs_intp)
                    if apn_ann[0] == 'N': # Normal
                        label = 0.0
                    elif apn_ann[0] == 'A': # Apnea
                        label = 1.0
                    else:
                        label = 2.0

                    test_input_array.append([rri_intp, qrs_intp, age[data_index], sex[data_index]])
                    test_label_array.append(label)
        except:
                hrv_module_error = 1

np.save('spaches_test.npy', test_input_array)
np.save('k.npy', test_label_array)

patient5


100%|██████████| 98/98 [00:00<00:00, 578.87it/s]

[ 160  416  683  936 1168 1628 1848 2064 2275 2483 2704 2960 3240 3539
 3848 4158 4778 5103 5424 5742]
[0.28015914 0.21684212 0.45352029 0.36280665 0.22612881 0.24812451
 0.33153516 0.38432143 0.55795168 0.43553693 0.20545441 0.25289537
 0.24748771 0.51494667 0.28525896 0.17696936 0.18721232 0.4813548
 0.29190563 0.37634661]
[256 267 253 232 460 220 216 211 208 221 256 280 299 309 310 620 325 321
 318]
[2560. 2670. 2530. 2320. 4600. 2200. 2160. 2110. 2080. 2210. 2560. 2800.
 2990. 3090. 3100. 6200. 3250. 3210. 3180.]
RRi array([2560., 2560., 2530., 2530., 2320., 2200., 2160., 2110., 2110.,
       2210., 2560., 2800., 2990., 3090., 3100., 3250., 3250., 3210.,
       3180.])
[ 6040  6331  6624  6936  7247  7531  7816  8104  8390  8667  8963  9264
  9563 10128 10416 10712 10995 11264 11532 11800]
[0.23422644 0.49669078 0.29203975 0.31667907 0.53452524 0.42792223
 0.2959527  0.30936064 0.40054931 0.34811205 0.49655139 0.38021797
 0.51124472 0.2336753  0.32918501 0.22503525 0.55551235 0.283

In [112]:
test_label_array


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0